# Task 1

## First, install the dependenices and download data

In [12]:
! pip install kaggle
! sudo ln -s ~/.local/bin/kaggle /usr/bin/kaggle
! sudo apt-get update && sudo apt-get install p7zip-full -y
! sudo pip3 install tqdm

ln: failed to create symbolic link '/usr/bin/kaggle': File exists
Hit:1 http://mirror.yandex.ru/ubuntu focal InRelease
Get:2 http://mirror.yandex.ru/ubuntu focal-updates InRelease [114 kB]          
Get:3 http://mirror.yandex.ru/ubuntu focal-backports InRelease [108 kB]        
Hit:4 http://storage.yandexcloud.net/dataproc/ci/trunk/80-d30331cee3848e39 focal InRelease
Get:5 http://mirror.yandex.ru/mirrors/postgresql focal-pgdg InRelease [91.6 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:7 https://repos.influxdata.com/ubuntu focal InRelease [7,019 B]            
Hit:8 https://repo.saltproject.io/py3/ubuntu/20.04/amd64/3002 focal InRelease  
Get:9 http://mirror.yandex.ru/ubuntu focal-updates/main amd64 Packages [2,372 kB]
Get:10 http://mirror.yandex.ru/ubuntu focal-updates/main i386 Packages [785 kB]
Get:11 http://mirror.yandex.ru/ubuntu focal-updates/main Translation-en [408 kB]
Get:12 http://mirror.yandex.ru/ubuntu focal-updates/main amd64 c-n-

In [11]:
! kaggle competitions download -c avito-context-ad-clicks -f VisitsStream.tsv.7z

100%|█████████████████████████████████████▉| 1.98G/1.99G [00:40<00:00, 55.0MB/s]
100%|██████████████████████████████████████| 1.99G/1.99G [00:40<00:00, 52.9MB/s]


In [13]:
! mkdir -p filefolder && 7z x VisitsStream.tsv.7z -ofilefolder/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 2132222657 bytes (2034 MiB)

Extracting archive: VisitsStream.tsv.7z
--
Path = VisitsStream.tsv.7z
Type = 7z
Physical Size = 2132222657
Headers Size = 136
Method = LZMA:24
Solid = -
Blocks = 1

      0% - VisitsStream.ts                        1% - VisitsStream.ts                        2% - VisitsStream.ts                        3% - VisitsStream.ts                        4% - VisitsStream.ts                        5% - VisitsStream.ts                        6% - VisitsStream.ts                        7% - VisitsStream.ts                        8% - VisitsStream.ts                        9% - VisitsStream.ts                       10% - VisitsStream.ts                       11% - VisitsStream.ts                       12% - VisitsStream

In [14]:
! ls filefolder/

VisitsStream.tsv


In [15]:
! cat filefolder/VisitsStream.tsv | head -n 10

UserID	IPID	AdID	ViewDate
59703	1259356	469877	2015-04-25 00:00:00.0
154389	1846749	27252551	2015-04-25 00:00:00.0
218628	2108380	31685325	2015-04-25 00:00:00.0
231535	837110	18827716	2015-04-25 00:00:00.0
282306	1654210	29363673	2015-04-25 00:00:00.0
295068	601505	588324	2015-04-25 00:00:00.0
501897	158476	4103261	2015-04-25 00:00:00.0
655394	631692	9860544	2015-04-25 00:00:00.0
765603	804403	29475627	2015-04-25 00:00:00.0
cat: write error: Broken pipe


## Now, write MapReduce code and test it on subset of data

In [122]:
! cat filefolder/VisitsStream.tsv | head -n 10000 > testfile.tsv
! sed -i -e '1'd testfile.tsv

cat: write error: Broken pipe


In [123]:
%%writefile longest_session.py

import sys
import csv
import re
from itertools import groupby
from datetime import datetime


def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def tsv_stream():
    return csv.reader(iter(sys.stdin.readline, ''), delimiter="\t")


def mapper():
    for row in tsv_stream():
        userid = row[0]
        date_str = row[3]
        print("{}+{}\t".format(userid, date_str))


def reducer():
    user_date, _ = next(sys.stdin).split("\t")
    user, date_str = user_date.split("+")
    last_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S.%f")
    duration = 0
    max_duration = 0
    for current_user_date, _ in kv_stream("\t"):
        current_user, current_date_str = current_user_date.split("+")
        current_date = datetime.strptime(current_date_str, "%Y-%m-%d %H:%M:%S.%f")
        
        if current_user != user:
            print("{}\t{}".format(user, max_duration))
            user = current_user
            last_date = current_date
            max_duration = 0
            duration = 0
        else:
            if (current_date - last_date).seconds <= 15 * 60:
                duration += (current_date - last_date).seconds
            else:
                duration = 0
            max_duration = max(max_duration, duration)
            last_date = current_date
    max_duration = max(max_duration, duration)
    print("{}\t{}".format(user, max_duration))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Overwriting longest_session.py


In [128]:
%%writefile top10.py

import sys
import collections
from itertools import islice

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for key, value in kv_stream():
        print("{}+{}\t".format(key, value.strip()))

def reducer():
    first_10_stream = islice(kv_stream("+"), 10)
    
    for user, duration in first_10_stream:
        print("{}\t{}".format(user, duration.strip()))
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Overwriting top10.py


In [129]:
%%time

! cat testfile.tsv | \
    tqdm --total $(cat testfile.tsv | wc -l) | \
    python longest_session.py map | \
    sort -t'+' -k1,1 -k2,2 | \
    python longest_session.py reduce > result.txt

100%|███████████████████████████████████| 9999/9999 [00:00<00:00, 661329.09it/s]
CPU times: user 623 µs, sys: 8.33 ms, total: 8.96 ms
Wall time: 309 ms


In [130]:
%%time

! cat result.txt | \
    tqdm --total $(cat result.txt | wc -l) | \
    python top10.py map | \
    sort -t'+' -k2,2nr -k1,1 | \
    python top10.py reduce > top-10.txt

100%|██████████████████████████████████| 5820/5820 [00:00<00:00, 1619723.26it/s]
CPU times: user 4.09 ms, sys: 3.14 ms, total: 7.23 ms
Wall time: 186 ms


In [127]:
! cat top-10.txt

1147902	160
365809	159
1316443	158
2560649	158
3093941	158
3783187	157
1435568	156
4285777	156
444510	156
218628	155


## Now, run the code on hdfs on the same subset of data.
Make sure that it works.

In [131]:
! hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2023-02-10 10:58 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2023-02-10 10:58 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2023-02-10 10:58 /user
drwxrwxrwt   - hdfs   hadoop          0 2023-02-10 10:58 /var


In [132]:
! hdfs dfs -rm -r /user/hw2/data
! hdfs dfs -mkdir -p /user/hw2/data

rm: `/user/hw2/data': No such file or directory


In [134]:
! hdfs dfs -put testfile.tsv /user/hw2/data/

put: `/user/hw2/data/testfile.tsv': File exists


In [135]:
! sudo mkdir -p /usr/lib/hadoop/logs
! sudo chmod 0777 /usr/lib/hadoop/logs
! sudo -u hdfs hdfs balancer 

2023-02-10 16:19:53,039 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net:8020]
2023-02-10 16:19:53,044 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 10.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2023-02-10 16:19:53,045 INFO balancer.Balancer: included nodes = []
2023-02-10 16:19:53,045 INFO balancer.Balancer: excluded nodes = []
2023-02-10 16:19:53,045 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2023-02-10 16:19:53,051 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
2023-02-10 16:19:54,239 INFO balancer.Balancer: dfs.namenode.get-blocks.max-qps = 20 (default=20)
2023-02-10 16:19:54,239 IN

In [138]:
! sudo find /usr/ -name hadoop-streaming.jar 

/usr/lib/hadoop-mapreduce/hadoop-streaming.jar


In [139]:
! sudo file /usr/lib/hadoop-mapreduce/hadoop-streaming.jar

/usr/lib/hadoop-mapreduce/hadoop-streaming.jar: symbolic link to hadoop-streaming-3.2.2.jar


In [140]:
%%time

! hdfs dfs -rm -r /user/hw2/result || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="longest-session" \
-D mapreduce.job.reduces=3 \
-files ~/longest_session.py \
-mapper "python3 longest_session.py map" \
-reducer "python3 longest_session.py reduce" \
-input /user/hw2/data/ \
-output /user/hw2/result/

rm: `/user/hw2/result': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob8625384959581521654.jar tmpDir=null
2023-02-10 16:23:48,728 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-10 16:23:49,068 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-10 16:23:49,121 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-10 16:23:49,122 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-10 16:23:49,417 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1676026733147_0001
2023-02-10 16:23:49,746 INFO mapred.FileInputFormat: T

In [141]:
%%time

! hdfs dfs -rm -r /user/hw2/top10/
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top-10" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files top10.py \
-mapper "python top10.py map" \
-reducer "python top10.py reduce" \
-input /user/hw2/result/ \
-output /user/hw2/top10/

rm: `/user/hw2/top10/': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob1610826423214658925.jar tmpDir=null
2023-02-10 16:25:47,215 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-10 16:25:47,456 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-10 16:25:47,498 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-10 16:25:47,499 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-10 16:25:47,750 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1676026733147_0002
2023-02-10 16:25:48,030 INFO mapred.FileInputFormat: T

## Fine! Now, we are ready to run the code on all data

In [142]:
! sed -i -e '1'd filefolder/VisitsStream.tsv

In [143]:
! hdfs dfs -rm -r /user/hw2/data
! hdfs dfs -mkdir -p /user/hw2/data
! hdfs dfs -put filefolder/VisitsStream.tsv /user/hw2/data/

Deleted /user/hw2/data


In [147]:
! hdfs dfs -ls /user/hw2/data/

Found 1 items
-rw-r--r--   1 ubuntu hadoop 13180996366 2023-02-10 16:50 /user/hw2/data/VisitsStream.tsv


In [148]:
! sudo -u hdfs hdfs balancer -threshold 1

2023-02-10 17:05:02,845 INFO balancer.Balancer: Using a threshold of 1.0
2023-02-10 17:05:02,847 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net:8020]
2023-02-10 17:05:02,849 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 1.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2023-02-10 17:05:02,849 INFO balancer.Balancer: included nodes = []
2023-02-10 17:05:02,849 INFO balancer.Balancer: excluded nodes = []
2023-02-10 17:05:02,849 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2023-02-10 17:05:02,857 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
2023-02-10 17:05:04,158 INFO balancer.Balancer: dfs.

In [149]:
! sudo -u hdfs hdfs dfsadmin -report

Configured Capacity: 211162595328 (196.66 GB)
Present Capacity: 177107763263 (164.94 GB)
DFS Remaining: 163820646400 (152.57 GB)
DFS Used: 13287116863 (12.37 GB)
DFS Used%: 7.50%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 10.128.0.26:9866 (rc1a-dataproc-d-lcng4r2l7s9xard5.mdb.yandexcloud.net)
Hostname: rc1a-dataproc-d-lcng4r2l7s9xard5.mdb.yandexcloud.net
Decommission Status : Normal
Configured Capacity: 105581297664 (98.33 GB)
DFS Used: 5682761728 (5.29 GB)
Non DFS Used: 12662398976 (11.79 GB)
DFS Remaining:

In [ ]:
%%time

! hdfs dfs -rm -r /user/hw2/result || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="longest-session" \
-D mapreduce.job.reduces=3 \
-files ~/longest_session.py \
-mapper "python3 longest_session.py map" \
-reducer "python3 longest_session.py reduce" \
-input /user/hw2/data/ \
-output /user/hw2/result/

Deleted /user/hw2/result
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob5200837507345007401.jar tmpDir=null
2023-02-10 17:05:51,152 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-10 17:05:51,636 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-10 17:05:51,693 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-10 17:05:51,694 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-10 17:05:52,032 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1676026733147_0003
2023-02-10 17:05:52,372 INFO mapred.FileInputFormat: Total input files to proce

In [151]:
! hdfs dfs -cat /user/hw2/result/* | head

1	540
100	199
1000	3022
10000	964
100000	0
1000000	0
1000001	937
1000002	0
1000003	1023
1000005	0
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


In [153]:
%%time

! hdfs dfs -rm -r /user/hw2/top10/
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top-10" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files top10.py \
-mapper "python top10.py map" \
-reducer "python top10.py reduce" \
-input /user/hw2/result/ \
-output /user/hw2/top10/

Deleted /user/hw2/top10
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob6669817881849552092.jar tmpDir=null
2023-02-11 09:03:01,637 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-11 09:03:01,864 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-11 09:03:01,898 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:8032
2023-02-11 09:03:01,899 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-4doa8g36b59o741x.mdb.yandexcloud.net/10.128.0.8:10200
2023-02-11 09:03:02,109 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1676026733147_0005
2023-02-11 09:03:02,405 INFO mapred.FileInputFormat: Total input files to proces

In [159]:
! hdfs dfs -cat /user/hw2/top10/part-00000

1113291	329463
1113291	329327
1113291	329327
1203081	109500
1203081	109474
1203081	109362
4263912	74135
4263912	73660
4263912	71505
1987990	70543


In [161]:
! hdfs dfs -ls /user/hw2/top10/

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2023-02-11 09:04 /user/hw2/top10/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        146 2023-02-11 09:04 /user/hw2/top10/part-00000


In [164]:
! hdfs dfs -cp /user/hw2/top10/part-00000 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt

2023-02-11 09:08:44,113 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-11 09:08:44,196 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-11 09:08:44,196 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2023-02-11 09:08:46,908 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-11 09:08:46,909 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-11 09:08:46,909 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [165]:
! hdfs dfs -ls s3a://lsml-sasha-bakalova-data/

2023-02-11 09:08:49,870 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-11 09:08:49,954 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-11 09:08:49,954 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 2 items
-rw-rw-rw-   1 ubuntu ubuntu        157 2023-01-23 09:16 s3a://lsml-sasha-bakalova-data/mhw1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        146 2023-02-11 09:08 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt
2023-02-11 09:08:51,438 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-11 09:08:51,438 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-11 09:08:51,439 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


## [link](https://storage.yandexcloud.net/lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt) to object storage

# Task 2

## Downloading data (we need other tables now)

In [1]:
! kaggle competitions download -c avito-context-ad-clicks

100%|██████████████████████████████████████| 17.8G/17.8G [07:27<00:00, 59.9MB/s]
100%|██████████████████████████████████████| 17.8G/17.8G [07:27<00:00, 42.6MB/s]


In [3]:
! sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 56 not upgraded.
Need to get 168 kB of archives.
After this operation, 593 kB of additional disk space will be used.
Get:1 http://mirror.yandex.ru/ubuntu focal-updates/main amd64 unzip amd64 6.0-25ubuntu1.1 [168 kB]
Fetched 168 kB in 0s (7,462 kB/s)
Selecting previously unselected package unzip.
(Reading database ... 126845 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-25ubuntu1.1_amd64.deb ...
Unpacking unzip (6.0-25ubuntu1.1) ...
Setting up unzip (6.0-25ubuntu1.1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
Processing triggers for man-db (2.9.1-1) ...


In [5]:
! yes | unzip avito-context-ad-clicks.zip

Archive:  avito-context-ad-clicks.zip
replace AdsInfo.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: AdsInfo.tsv.7z          
replace Category.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: Category.tsv.7z         
replace Location.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: Location.tsv.7z         
replace PhoneRequestsStream.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: PhoneRequestsStream.tsv.7z  
replace SearchInfo.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: SearchInfo.tsv.7z       
replace UserInfo.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: UserInfo.tsv.7z         
replace VisitsStream.tsv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: VisitsStream.tsv.7z     
  inflating: database.sqlite.7z      
  inflating: sampleSubmission.csv.7z  
  inflating: sampleSubmission_HistCTR.csv.7z  
  inflating: testSearchStream.tsv.7z  
  inflating: trainSearchStream.tsv.7z  
yes: standard output: Broken pip

In [6]:
! mkdir -p hw2folder2 && 7z x VisitsStream.tsv.7z -ohw2folder2/
! 7z x trainSearchStream.tsv.7z -ohw2folder2/
! 7z x UserInfo.tsv.7z -ohw2folder2/
! 7z x SearchInfo.tsv.7z -ohw2folder2/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 2132222657 bytes (2034 MiB)

Extracting archive: VisitsStream.tsv.7z
--
Path = VisitsStream.tsv.7z
Type = 7z
Physical Size = 2132222657
Headers Size = 136
Method = LZMA:24
Solid = -
Blocks = 1

      0% - VisitsStream.ts                        1% - VisitsStream.ts                        2% - VisitsStream.ts                        3% - VisitsStream.ts                        4% - VisitsStream.ts                        5% - VisitsStream.ts                        6% - VisitsStream.ts                        7% - VisitsStream.ts                        8% - VisitsStream.ts                        9% - VisitsStream.ts                       10% - VisitsStream.ts                       11% - VisitsStream.ts                       12% - VisitsStream

In [7]:
! pip3 install findspark

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
sc = pyspark.SparkContext(appName="lsml-app")

### Task 2.1

In [ ]:
! hdfs dfs -put hw2folder2/SearchInfo.tsv /user/hw2/data/

In [8]:
search_info = sc.textFile("/user/hw2/data/SearchInfo.tsv")

In [21]:
import re

def get_filters_with_dots(searchparams):
    if searchparams == "" or searchparams == 'SearchParams':
        return []
    pattern = re.compile(r"[0-9]+:")
    return pattern.findall(s)

result = (
    search_info
    .map(lambda row : row.split("\t")[-1])
    .flatMap(get_filters_with_dots)
    .map(lambda x : (int(x[:-1]), 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

In [26]:
result

[(797, 53562267),
 (5, 53562267),
 (799, 53562267),
 (800, 53562267),
 (801, 53562267),
 (709, 53562267)]

In [34]:
import json
with open("hw2/subtask1.txt", "w") as file:
    file.write('\n'.join(list(map(lambda x : str(x[0]), result))))

In [35]:
! cat "hw2/subtask1.txt"

797
5
799
800
801
709

In [70]:
! hdfs dfs -put hw2/subtask1.txt s3a://lsml-sasha-bakalova-data/mhw2-subtask1-output.txt

2023-02-11 23:49:29,770 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-11 23:49:30,119 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-11 23:49:30,120 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2023-02-11 23:49:33,559 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-11 23:49:33,559 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-11 23:49:33,559 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [71]:
! hdfs dfs -ls s3a://lsml-sasha-bakalova-data/

2023-02-11 23:49:38,240 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-11 23:49:38,317 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-11 23:49:38,317 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 3 items
-rw-rw-rw-   1 ubuntu ubuntu        157 2023-01-23 09:16 s3a://lsml-sasha-bakalova-data/mhw1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        146 2023-02-11 09:08 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt
-rw-rw-rw-   1 ubuntu ubuntu         21 2023-02-11 23:49 s3a://lsml-sasha-bakalova-data/mhw2-subtask1-output.txt
2023-02-11 23:49:39,790 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-11 23:49:39,790 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-11 23:49:39,790 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


### [link](https://storage.yandexcloud.net/lsml-sasha-bakalova-data/mhw2-subtask1-output.txt) to object storage

### Task 2.2

In [47]:
! hdfs dfs -put hw2folder2/trainSearchStream.tsv /user/hw2/data/

In [50]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [51]:
search_info_df = se.read.csv("/user/hw2/data/SearchInfo.tsv", sep="\t", header=True, inferSchema=True)

In [53]:
search_info_df.show()

+--------+--------------------+-------+-------+--------------+-----------+----------+----------+--------------------+
|SearchID|          SearchDate|   IPID| UserID|IsUserLoggedOn|SearchQuery|LocationID|CategoryID|        SearchParams|
+--------+--------------------+-------+-------+--------------+-----------+----------+----------+--------------------+
|       1|2015-05-18 19:54:...|1717090|3640266|             0|       null|      1729|         5|                null|
|       2|2015-05-12 14:21:...|1731568| 769304|             0|       null|       697|        50|                null|
|       3|2015-05-12 07:09:...| 793143| 640089|             0|       null|      1261|        12|                null|
|       4|2015-05-10 18:11:...| 898705|3573776|             0|       null|      3960|        22|{83:'Обувь', 175:...|
|       5|2015-04-25 13:04:...|2009707| 320674|             0|       null|       547|         1|                null|
|       6|2015-05-07 16:49:...|1658456|1665156|         

In [52]:
search_stream_df = se.read.csv("/user/hw2/data/trainSearchStream.tsv", sep="\t", header=True, inferSchema=True)

In [54]:
search_stream_df.show()

+--------+--------+--------+----------+--------+-------+
|SearchID|    AdID|Position|ObjectType| HistCTR|IsClick|
+--------+--------+--------+----------+--------+-------+
|       2|11441863|       1|         3|0.001804|      0|
|       2|22968355|       7|         3|0.004723|      0|
|       3|  212187|       7|         3|0.029701|      0|
|       3|34084553|       1|         3|  0.0043|      0|
|       3|36256251|       2|         2|    null|   null|
|       4| 2073399|       6|         1|    null|   null|
|       4| 6046052|       7|         1|    null|   null|
|       4|17544913|       8|         1|    null|   null|
|       4|20653823|       1|         3|0.003049|      0|
|       4|24129570|       2|         2|    null|   null|
|       5|  530202|       6|         1|    null|   null|
|       5|11219482|       1|         3|0.043897|      0|
|       5|13375896|       7|         3|0.001563|      0|
|       5|30074367|       2|         2|    null|   null|
|       5|33506654|       8|   

In [55]:
search_info_df.registerTempTable('search_info')
search_stream_df.registerTempTable('search_stream') 

In [57]:
words = se.sql("""
    SELECT SearchQuery
    FROM search_info si
        inner join search_stream ss on ss.SearchID = si.SearchID
    WHERE ss.IsClick = 1
""")

In [58]:
words.show()

+--------------------+
|         SearchQuery|
+--------------------+
|                null|
|                null|
|             монопод|
|                null|
|                null|
|                null|
|сварочный аппарат бу|
|                null|
|     iphone 5s новый|
|            ваз 2110|
|               туфли|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
+--------------------+
only showing top 20 rows



In [74]:
import string
def transform_query(query):
    if query is None:
        return []
    return ["".join([i if i not in string.punctuation else "" for i in w]) for w in query.lower().split()]

result = (
    words.rdd
    .map(lambda x : x.SearchQuery)
    .flatMap(transform_query)
    .map(lambda x : (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

In [75]:
result

[('бу', 12599),
 ('велосипед', 12510),
 ('iphone', 7442),
 ('для', 5984),
 ('диван', 5716),
 ('на', 4905),
 ('платье', 4465),
 ('коляска', 3785),
 ('велосипеды', 3780),
 ('самокат', 3447)]

In [76]:
import json
with open("hw2/subtask2.txt", "w") as file:
    file.write('\n'.join(list(map(lambda x : f"{x[0]}\t{x[1]}", result))))
! cat "hw2/subtask2.txt"

бу	12599
велосипед	12510
iphone	7442
для	5984
диван	5716
на	4905
платье	4465
коляска	3785
велосипеды	3780
самокат	3447

In [77]:
! hdfs dfs -put hw2/subtask2.txt s3a://lsml-sasha-bakalova-data/mhw2-subtask2-output.txt

2023-02-11 23:53:10,575 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-11 23:53:10,659 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-11 23:53:10,660 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2023-02-11 23:53:12,678 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-11 23:53:12,678 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-11 23:53:12,678 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [78]:
! hdfs dfs -ls s3a://lsml-sasha-bakalova-data/

2023-02-11 23:53:13,879 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-11 23:53:13,950 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-11 23:53:13,950 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 4 items
-rw-rw-rw-   1 ubuntu ubuntu        157 2023-01-23 09:16 s3a://lsml-sasha-bakalova-data/mhw1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        146 2023-02-11 09:08 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt
-rw-rw-rw-   1 ubuntu ubuntu         21 2023-02-11 23:49 s3a://lsml-sasha-bakalova-data/mhw2-subtask1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        169 2023-02-11 23:53 s3a://lsml-sasha-bakalova-data/mhw2-subtask2-output.txt
2023-02-11 23:53:15,445 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-11 23:53:15,445 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-11 23:53:15,445 INFO impl.MetricsSystemImpl: s3a-fi

### [link](https://storage.yandexcloud.net/lsml-sasha-bakalova-data/mhw2-subtask2-output.txt) to object storage

### Task 2.3

In [1]:
! hdfs dfs -put hw2folder2/VisitsStream.tsv /user/hw2/data/

put: `/user/hw2/data/VisitsStream.tsv': File exists


In [5]:
visits_stream = sc.textFile("/user/hw2/data/VisitsStream.tsv")

In [6]:
# already got rid of the header in the first task
visits_stream.take(10)

['59703\t1259356\t469877\t2015-04-25 00:00:00.0',
 '154389\t1846749\t27252551\t2015-04-25 00:00:00.0',
 '218628\t2108380\t31685325\t2015-04-25 00:00:00.0',
 '231535\t837110\t18827716\t2015-04-25 00:00:00.0',
 '282306\t1654210\t29363673\t2015-04-25 00:00:00.0',
 '295068\t601505\t588324\t2015-04-25 00:00:00.0',
 '501897\t158476\t4103261\t2015-04-25 00:00:00.0',
 '655394\t631692\t9860544\t2015-04-25 00:00:00.0',
 '765603\t804403\t29475627\t2015-04-25 00:00:00.0',
 '790289\t121085\t23309988\t2015-04-25 00:00:00.0']

In [9]:
# get number of days in the dataset

from datetime import datetime

def get_date(row):
    date = datetime.strptime(row.split("\t")[-1], "%Y-%m-%d %H:%M:%S.%f")
    return (date.year, date.month, date.day)

(
    visits_stream
    .map(get_date)
    .distinct()
    .count()
)

26

In [11]:
from datetime import datetime

TOTAL_NUM_DAYS = 26 # result of the previous calculation

def get_user_and_date(row):
    date = datetime.strptime(row.split("\t")[-1], "%Y-%m-%d %H:%M:%S.%f")
    return (row.split("\t")[0], f"{date.year} {date.month} {date.day}")

(
    visits_stream
    .map(get_user_and_date)
    .distinct()
    .map(lambda x : (x[0], 1))
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x : x[1] == TOTAL_NUM_DAYS)
    .saveAsTextFile("hw2/everyday_users.txt")
)

In [31]:
everyday_users = sc.textFile("hw2/everyday_users.txt").map(lambda x : eval(x)[0])
everyday_users.take(10)

['3003794',
 '981692',
 '3230519',
 '3296369',
 '1247227',
 '1243244',
 '1407121',
 '2266007',
 '1212571',
 '2539404']

In [24]:
result = everyday_users.count()

In [25]:
result

807

In [27]:
import json
with open("hw2/subtask3.txt", "w") as file:
    file.write(str(result))
! cat "hw2/subtask3.txt"

807

In [28]:
! hdfs dfs -put hw2/subtask3.txt s3a://lsml-sasha-bakalova-data/mhw2-subtask3-output.txt

2023-02-12 14:22:04,372 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-12 14:22:04,472 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-12 14:22:04,472 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2023-02-12 14:22:06,855 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-12 14:22:06,855 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-12 14:22:06,855 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [29]:
! hdfs dfs -ls s3a://lsml-sasha-bakalova-data/

2023-02-12 14:22:08,018 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-12 14:22:08,105 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-12 14:22:08,105 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 5 items
-rw-rw-rw-   1 ubuntu ubuntu        157 2023-01-23 09:16 s3a://lsml-sasha-bakalova-data/mhw1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        146 2023-02-11 09:08 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt
-rw-rw-rw-   1 ubuntu ubuntu         21 2023-02-11 23:49 s3a://lsml-sasha-bakalova-data/mhw2-subtask1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        169 2023-02-11 23:53 s3a://lsml-sasha-bakalova-data/mhw2-subtask2-output.txt
-rw-rw-rw-   1 ubuntu ubuntu          3 2023-02-12 14:22 s3a://lsml-sasha-bakalova-data/mhw2-subtask3-output.txt
2023-02-12 14:22:09,607 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-12 14:22:09,607 INFO impl.Metri

### [link](https://storage.yandexcloud.net/lsml-sasha-bakalova-data/mhw2-subtask3-output.txt) to object storage

### Task 2.4

In [32]:
# it is okay, because we have only 807 users in this list
everyday_users_list = everyday_users.collect()

In [54]:
visits_stream.take(3)

['59703\t1259356\t469877\t2015-04-25 00:00:00.0',
 '154389\t1846749\t27252551\t2015-04-25 00:00:00.0',
 '218628\t2108380\t31685325\t2015-04-25 00:00:00.0']

In [55]:
from datetime import datetime

def get_date_and_user(row):
    date = datetime.strptime(row.split("\t")[-1], "%Y-%m-%d %H:%M:%S.%f")
    return (f"{date.year}-{date.month}-{date.day}", row.split("\t")[0])

def swap_key_value(kv):
    return (kv[1], kv[0])


# we can call collect because there are only 26 distinct days in the dataset
result = (
    visits_stream
    .map(get_date_and_user)
    .filter(lambda x : str(x[1]) not in everyday_users_list)
    .distinct()
    .map(lambda x : (x[0], 1))
    .reduceByKey(lambda a, b: a + b)
    .map(swap_key_value)
    .sortByKey(ascending=False)
    .map(swap_key_value)
    .collect()
)

In [56]:
result

[('2015-5-12', 973248),
 ('2015-5-13', 847446),
 ('2015-5-14', 745904),
 ('2015-5-6', 733027),
 ('2015-5-5', 731624),
 ('2015-5-7', 729708),
 ('2015-5-11', 706240),
 ('2015-4-27', 676433),
 ('2015-5-8', 664982),
 ('2015-4-28', 655791),
 ('2015-4-29', 641148),
 ('2015-5-15', 633132),
 ('2015-5-4', 607205),
 ('2015-4-30', 596224),
 ('2015-5-10', 562151),
 ('2015-4-26', 540051),
 ('2015-5-3', 511687),
 ('2015-4-25', 498069),
 ('2015-5-16', 493577),
 ('2015-5-18', 488250),
 ('2015-5-17', 480968),
 ('2015-5-2', 468010),
 ('2015-5-1', 452800),
 ('2015-5-9', 450135),
 ('2015-5-19', 367777),
 ('2015-5-20', 184972)]

In [57]:
import json
with open("hw2/subtask4.txt", "w") as file:
    file.write("\n".join(map(lambda x : f"{x[0]}\t{x[1]}", result)))
! cat "hw2/subtask4.txt"

2015-5-12	973248
2015-5-13	847446
2015-5-14	745904
2015-5-6	733027
2015-5-5	731624
2015-5-7	729708
2015-5-11	706240
2015-4-27	676433
2015-5-8	664982
2015-4-28	655791
2015-4-29	641148
2015-5-15	633132
2015-5-4	607205
2015-4-30	596224
2015-5-10	562151
2015-4-26	540051
2015-5-3	511687
2015-4-25	498069
2015-5-16	493577
2015-5-18	488250
2015-5-17	480968
2015-5-2	468010
2015-5-1	452800
2015-5-9	450135
2015-5-19	367777
2015-5-20	184972

In [58]:
! hdfs dfs -put hw2/subtask4.txt s3a://lsml-sasha-bakalova-data/mhw2-subtask4-output.txt

2023-02-12 16:03:20,149 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-12 16:03:20,230 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-12 16:03:20,230 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2023-02-12 16:03:22,067 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-12 16:03:22,067 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-12 16:03:22,067 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [59]:
! hdfs dfs -ls s3a://lsml-sasha-bakalova-data/

2023-02-12 16:03:23,271 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-12 16:03:23,346 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-12 16:03:23,346 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 6 items
-rw-rw-rw-   1 ubuntu ubuntu        157 2023-01-23 09:16 s3a://lsml-sasha-bakalova-data/mhw1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        146 2023-02-11 09:08 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt
-rw-rw-rw-   1 ubuntu ubuntu         21 2023-02-11 23:49 s3a://lsml-sasha-bakalova-data/mhw2-subtask1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        169 2023-02-11 23:53 s3a://lsml-sasha-bakalova-data/mhw2-subtask2-output.txt
-rw-rw-rw-   1 ubuntu ubuntu          3 2023-02-12 14:22 s3a://lsml-sasha-bakalova-data/mhw2-subtask3-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        432 2023-02-12 16:03 s3a://lsml-sasha-bakalova-data/mhw2-subtask4-output.txt
2023-02-12 16:03:24,83

### [link](https://storage.yandexcloud.net/lsml-sasha-bakalova-data/mhw2-subtask4-output.txt) to object storage

### Task 2.5

In [4]:
search_info = sc.textFile("/user/hw2/data/SearchInfo.tsv")

In [5]:
! hdfs dfs -put hw2folder2/UserInfo.tsv /user/hw2/data/

In [6]:
user_info = sc.textFile("/user/hw2/data/UserInfo.tsv")

In [19]:
! cat hw2folder2/UserInfo.tsv | head
! cat hw2folder2/SearchInfo.tsv | head

UserID	UserAgentID	UserAgentOSID	UserDeviceID	UserAgentFamilyID
1	44073	30	2019	9
2	12505	20	2014	85
3	24256	20	2014	64
4	57133	20	2014	25
5	57133	20	2014	25
6	57201	20	2014	85
7	22293	20	2014	25
8	19983	35	3870	63
9	12505	20	2014	85
cat: write error: Broken pipe
SearchID	SearchDate	IPID	UserID	IsUserLoggedOn	SearchQuery	LocationID	CategoryID	SearchParams
1	2015-05-18 19:54:32.0	1717090	3640266	0		1729	5	
2	2015-05-12 14:21:28.0	1731568	769304	0		697	50	
3	2015-05-12 07:09:42.0	793143	640089	0		1261	12	
4	2015-05-10 18:11:01.0	898705	3573776	0		3960	22	{83:'Обувь', 175:'Женская одежда', 88:'38'}
5	2015-04-25 13:04:09.0	2009707	320674	0		547	1	
6	2015-05-07 16:49:15.0	1658456	1665156	0		926	27	
7	2015-05-14 23:07:27.0	1849117	3434614	0		44	500001	
8	2015-05-09 09:10:06.0	572585	905821	0		2694	12	
9	2015-05-02 20:14:15.0	77162	1106541	0		576	4	
cat: write error: Broken pipe


In [79]:
from math import sqrt

def get_relevant_rows_user_info_user_first(row):
    splitted = row.split("\t")
    if splitted[0] == "UserID":
        return []
    return [(splitted[0], splitted[2])]

def get_relevant_rows_search_info_user_first(row):
    splitted = row.split("\t")
    if splitted[0] == "SearchID":
        return []
    return [(splitted[3], splitted[7])]

def get_relevant_rows_user_info(row):
    splitted = row.split("\t")
    if splitted[0] == "UserID":
        return []
    return [(splitted[2], splitted[0])]

def get_relevant_rows_search_info(row):
    splitted = row.split("\t")
    if splitted[0] == "SearchID":
        return []
    return [(splitted[7], splitted[3])]

def get_category_to_count(row):
    splitted = row.split("\t")
    if splitted[0] == "SearchID":
        return []
    return [(splitted[7], 1)]

def get_userosid_to_count(row):
    splitted = row.split("\t")
    if splitted[0] == "UserID":
        return []
    return [(splitted[2], 1)]

def userid_first(row):
    key1, (userid, cnt) = row
    return userid, (key1, cnt)

def get_correlation(cnt_1, cnt_2, cnt_both, cnt_total):
    up = cnt_both - 2 * cnt_1 * cnt_2 / cnt_total + cnt_1 * cnt_2 / (cnt_total ** 2)
    down = sqrt((cnt_1 - cnt_1 ** 2 / cnt_total) * (cnt_2 - cnt_2 ** 2 / cnt_total))
    return up / down

DATASET_LEN = (
    search_info
        .flatMap(get_relevant_rows_search_info_user_first)
        .join(
            user_info
            .flatMap(get_relevant_rows_user_info_user_first)
        )
    .count()
)

result = ( # (catid, userosid)
    search_info
    .flatMap(get_relevant_rows_search_info)
    .join(
        search_info
        .flatMap(get_category_to_count)
        .reduceByKey(lambda a, b : a + b)
    )
    .map(userid_first)
    .join(
        user_info
        .flatMap(get_relevant_rows_user_info)
        .join(
            user_info
            .flatMap(get_userosid_to_count)
            .reduceByKey(lambda a, b : a + b)
        )
        .map(userid_first)
    )
    .map(lambda row: ((row[1][0][0], row[1][1][0]), (row[1][0][1], row[1][1][1])))# (cat, userosid), (#cat, #userosid)
    .join(
        search_info
        .flatMap(get_relevant_rows_search_info_user_first)
        .join(
            user_info
            .flatMap(get_relevant_rows_user_info_user_first)
        )
        .map(lambda row : (row[1], 1))
        .reduceByKey(lambda a, b : a + b)
    )
    .map(lambda row : (row[0], (row[1][0][0], row[1][0][1], row[1][1]))) # (cat, userosid), (#cat, #userosid, #cat_and_userosid)
    .map(lambda row : (row[0], get_correlation(*row[1], DATASET_LEN)))
    .distinct()
    .takeOrdered(10, lambda x : -x[1])
)

In [80]:
result

[(('34', '20'), 1.3631589353908604),
 (('22', '20'), 1.2870800179055524),
 (('38', '20'), 1.2185713707280852),
 (('12', '20'), 1.1098459756153582),
 (('60', '20'), 1.0088773311553494),
 (('47', '20'), 1.0058198235530058),
 (('0', '20'), 0.9646020438376409),
 (('41', '20'), 0.8445054700158525),
 (('50', '20'), 0.8124688986998632),
 (('26', '20'), 0.7245170642697342)]

In [81]:
import json
with open("hw2/subtask5.txt", "w") as file:
    file.write("\n".join(map(lambda x : f"{x[0][1]}\t{x[0][0]}\t{x[1]}", result)))
! cat "hw2/subtask5.txt"

20	34	1.3631589353908604
20	22	1.2870800179055524
20	38	1.2185713707280852
20	12	1.1098459756153582
20	60	1.0088773311553494
20	47	1.0058198235530058
20	0	0.9646020438376409
20	41	0.8445054700158525
20	50	0.8124688986998632
20	26	0.7245170642697342

In [82]:
! hdfs dfs -put hw2/subtask5.txt s3a://lsml-sasha-bakalova-data/mhw2-subtask5-output.txt

2023-02-13 13:00:38,508 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-13 13:00:38,587 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-13 13:00:38,587 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2023-02-13 13:00:41,560 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2023-02-13 13:00:41,560 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2023-02-13 13:00:41,560 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [83]:
! hdfs dfs -ls s3a://lsml-sasha-bakalova-data/

2023-02-13 13:00:42,888 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-02-13 13:00:42,973 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-02-13 13:00:42,973 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 7 items
-rw-rw-rw-   1 ubuntu ubuntu        157 2023-01-23 09:16 s3a://lsml-sasha-bakalova-data/mhw1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        146 2023-02-11 09:08 s3a://lsml-sasha-bakalova-data/mhw2-mapreduce-output.txt
-rw-rw-rw-   1 ubuntu ubuntu         21 2023-02-11 23:49 s3a://lsml-sasha-bakalova-data/mhw2-subtask1-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        169 2023-02-11 23:53 s3a://lsml-sasha-bakalova-data/mhw2-subtask2-output.txt
-rw-rw-rw-   1 ubuntu ubuntu          3 2023-02-12 14:22 s3a://lsml-sasha-bakalova-data/mhw2-subtask3-output.txt
-rw-rw-rw-   1 ubuntu ubuntu        432 2023-02-12 16:03 s3a://lsml-sasha-bakalova-data/mhw2-subtask4-output.txt
-rw-rw-rw-   1 ubuntu 

### [link](https://storage.yandexcloud.net/lsml-sasha-bakalova-data/mhw2-subtask5-output.txt) to object storage